**Importing the data from Google Drive**

In [2]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%%capture
# Unzip the data files
!unzip /content/drive/My\ Drive/chest-xray-pneumonia.zip

**Defining the data sources and creating the generators**

In [0]:
# Define the data sources
train_dir = '/content/chest_xray/train'
val_dir = '/content/chest_xray/val'
test_dir = '/content/chest_xray/test'

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator( )
test_datagen = ImageDataGenerator( )

# Create batches
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    batch_size = 12,
                                                    class_mode = 'binary',
                                                    shuffle = True,
                                                    target_size = (2090,1858))

val_generator = train_datagen.flow_from_directory(val_dir, 
                                                    batch_size = 12,
                                                    class_mode = 'binary',
                                                    target_size = (2090,1858))

test_generator = test_datagen.flow_from_directory(test_dir, 
                                                    batch_size = 12,
                                                    class_mode = 'binary',
                                                    target_size = (2090,1858))

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


**Building the CNN model in keras**

In [5]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', input_shape = (2090,1858,3), strides=(4, 4)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 522, 464, 16)      784       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 261, 232, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 129, 115, 16)      4112      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 57, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 27, 16)        4112      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 13, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 5, 32)          8

In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer = RMSprop (lr = 0.001),
             loss = 'binary_crossentropy',
             metrics = ['acc'])

**Training the model**

In [7]:
history = model.fit(train_generator, 
                              validation_data = val_generator,
                              steps_per_epoch = len(train_generator),
                              epochs = 5,
                              validation_steps = len(val_generator),
                              verbose = 1)

Epoch 1/5
435/435 [==============================] - 312s 717ms/step - loss: 0.4782 - acc: 0.8499 - val_loss: 1.0732 - val_acc: 0.6875
Epoch 2/5
435/435 [==============================] - 309s 711ms/step - loss: 0.2325 - acc: 0.9149 - val_loss: 0.5995 - val_acc: 0.8125
Epoch 3/5
435/435 [==============================] - 310s 713ms/step - loss: 0.1550 - acc: 0.9421 - val_loss: 0.2395 - val_acc: 0.9375
Epoch 4/5
435/435 [==============================] - 294s 676ms/step - loss: 0.1423 - acc: 0.9528 - val_loss: 0.4849 - val_acc: 0.8750
Epoch 5/5
435/435 [==============================] - 292s 671ms/step - loss: 0.1160 - acc: 0.9605 - val_loss: 0.2138 - val_acc: 0.9375


We used a small number of epochs to see what happens and to keep the training time low.
Nevertheless, he model seems to be super performant.
Let's see what happens when we use it to predict the test dataset.



**Evaluating the performances**

In [8]:
# evaluate loaded model on test data
score = model.evaluate(test_generator, steps = len(test_generator), verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

52/52 [==============================] - 33s 631ms/step - loss: 2.3658 - acc: 0.6987
acc: 69.87%


The results are not so good, especially compared to results on the validation data. We will shuffle these sets to see how it affects the performance.
Moreover, the validation set provided contains only 16 images, which seems to be too small compared to the training and test.
We are going to move some files around to adjust the proportions.

In [0]:
import os
def move_from_to_n(src_dir,dst_dir, n):
    # move files, no duplicates!
    fs = os.listdir(src_dir)
    assert len(fs) >= n
    i=0
    targ = fs[-n:]
    # print('targ', targ)
    for f in targ:
        # print('move_from_to_n', i, f, src_dir, dst_dir)
        os.rename(src_dir+'/'+f, dst_dir+'/'+f)
        i+=1
    print('copied',i,src_dir,dst_dir)

In [0]:
%%capture
# we move files from the training dataset to the validation one
move_from_to_n('/content/chest_xray/train/NORMAL', '/content/chest_xray/val/NORMAL', 150)
move_from_to_n('/content/chest_xray/train/PNEUMONIA', '/content/chest_xray/val/PNEUMONIA', 150)

move_from_to_n('/content/chest_xray/test/NORMAL', '/content/chest_xray/val/NORMAL', 150)
move_from_to_n('/content/chest_xray/test/PNEUMONIA', '/content/chest_xray/val/PNEUMONIA', 150)

move_from_to_n('/content/chest_xray/train/NORMAL', '/content/chest_xray/test/NORMAL', 150)
move_from_to_n('/content/chest_xray/train/PNEUMONIA', '/content/chest_xray/test/PNEUMONIA', 150)

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen_new = ImageDataGenerator( )
test_datagen_new = ImageDataGenerator( )

# Create batches
train_generator_new = train_datagen_new.flow_from_directory(train_dir, 
                                                    batch_size = 12,
                                                    class_mode = 'binary',
                                                    shuffle = True,
                                                    target_size = (2090,1858))

val_generator_new = train_datagen_new.flow_from_directory(val_dir, 
                                                    batch_size = 12,
                                                    class_mode = 'binary',
                                                    target_size = (2090,1858))

test_generator_new = test_datagen_new.flow_from_directory(test_dir, 
                                                    batch_size = 12,
                                                    class_mode = 'binary',
                                                    target_size = (2090,1858))

Found 4616 images belonging to 2 classes.
Found 616 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


To see the effect of this modification, let's see the training results with the same model.

In [0]:
model_new_data = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', input_shape = (2090,1858,3), strides=(4, 4)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [0]:
model_new_data.compile(optimizer = RMSprop (lr = 0.001),
             loss = 'binary_crossentropy',
             metrics = ['acc'])

In [25]:
history = model_new_data.fit(train_generator_new, 
                              validation_data = val_generator_new,
                              steps_per_epoch = len(train_generator_new),
                              epochs = 5,
                              validation_steps = len(val_generator_new),
                              verbose = 1)

Epoch 1/5
385/385 [==============================] - 305s 792ms/step - loss: 0.4644 - acc: 0.8559 - val_loss: 0.7963 - val_acc: 0.6834
Epoch 2/5
385/385 [==============================] - 305s 792ms/step - loss: 0.1905 - acc: 0.9307 - val_loss: 0.4573 - val_acc: 0.8604
Epoch 3/5
385/385 [==============================] - 310s 805ms/step - loss: 0.1542 - acc: 0.9435 - val_loss: 0.7497 - val_acc: 0.8068
Epoch 4/5
385/385 [==============================] - 308s 799ms/step - loss: 0.1352 - acc: 0.9526 - val_loss: 0.4936 - val_acc: 0.8360
Epoch 5/5
385/385 [==============================] - 312s 809ms/step - loss: 0.1325 - acc: 0.9601 - val_loss: 1.0279 - val_acc: 0.8003


Let's put some more parameters into the training process to make the best out of it.

In [0]:
import tensorflow as tf

model_new_data_pm = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', input_shape = (2090,1858,3), strides=(4, 4)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [0]:
from tensorflow.keras.optimizers import RMSprop

model_new_data_pm.compile(optimizer = RMSprop (lr = 0.001),
             loss = 'binary_crossentropy',
             metrics = ['acc'])

In [0]:
# Here we define when to stop the training, the version of the model to keep at the end (checkpoint) 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
earlyStopping = EarlyStopping(monitor='val_loss', patience=6, verbose=0, mode='min')
mcp_save = ModelCheckpoint('model.h5', save_best_only=True, monitor='val_loss', mode='min')

In [17]:
# We raise the number of epochs to let the other parameters do their job
history = model_new_data_pm.fit(train_generator_new, 
                              validation_data = val_generator_new,
                              steps_per_epoch = len(train_generator_new),
                              epochs = 50,
                              validation_steps = len(val_generator_new),
                              callbacks=[earlyStopping, mcp_save],
                              verbose = 1)

Epoch 1/50
385/385 [==============================] - 299s 776ms/step - loss: 0.9061 - acc: 0.8074 - val_loss: 0.6148 - val_acc: 0.7581
Epoch 2/50
385/385 [==============================] - 316s 820ms/step - loss: 0.2836 - acc: 0.8852 - val_loss: 0.5264 - val_acc: 0.8701
Epoch 3/50
385/385 [==============================] - 325s 845ms/step - loss: 0.2094 - acc: 0.9198 - val_loss: 0.5210 - val_acc: 0.9058
Epoch 4/50
385/385 [==============================] - 316s 820ms/step - loss: 0.1858 - acc: 0.9309 - val_loss: 0.5524 - val_acc: 0.8653
Epoch 5/50
385/385 [==============================] - 313s 813ms/step - loss: 0.1576 - acc: 0.9474 - val_loss: 1.3954 - val_acc: 0.7208
Epoch 6/50
385/385 [==============================] - 294s 763ms/step - loss: 0.1448 - acc: 0.9530 - val_loss: 0.7230 - val_acc: 0.8474
Epoch 7/50
385/385 [==============================] - 294s 763ms/step - loss: 0.1107 - acc: 0.9627 - val_loss: 1.7176 - val_acc: 0.7922
Epoch 8/50
385/385 [============================

**Saving the model and evaluating it**

In [18]:
# serialize model to JSON
model_json = model_new_data_pm.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
#model.save_weights("model.h5")

print("Saved model to disk")

Saved model to disk


In [19]:
from tensorflow.keras.models import model_from_json
# load json and create model
json_file = open('/content/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/model.h5")
print("Loaded model from disk")

Loaded model from disk


In [0]:
# Compile the model
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

**Evaluating the models**

In [21]:
# evaluate loaded model on test data
score = loaded_model.evaluate(test_generator_new, steps = len(test_generator_new), verbose=1)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

52/52 [==============================] - 35s 675ms/step - loss: 0.3818 - accuracy: 0.9054
accuracy: 90.54%


In [22]:
# evaluate the model fully trained (without checkpoint) on test data
score = model_new_data_pm.evaluate(test_generator_new, steps = len(test_generator_new), verbose=1)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

52/52 [==============================] - 35s 676ms/step - loss: 0.9924 - acc: 0.8462
accuracy: 84.62%
